# Recommend by Similarity

Finds similar recipes based on the name given by the user or both the name and calories and suggests the 5 closest recipes

Provides the recipe, ingredients and calorie information as output.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class RecipeRecommendationSystem:
    def __init__(self, data_path):
        self.df = pd.read_csv(data_path)
        self.preprocess_data()
        self.create_tfidf_matrix()
    
    def preprocess_data(self):
        """
        Okunan veriye veri ön işleme işlemleri gerçekleştiriliyor.
        """
        # Tarifle ilgili gerekli değişkenler seçiliyor
        text_columns = ['Name', 'RecipeCategory', 'Keywords', 'RecipeIngredientParts']
        self.df[text_columns] = self.df[text_columns].fillna('')
        
        # Seçilen değişkenler tek bir değişkende toplanıyor. Benzerlik bulmak için bu değişken kullanılacak
        self.df['combined_features'] = self.df['Name'] + ' ' + self.df['RecipeCategory'] + ' ' + \
                                       self.df['Keywords'] + ' ' + self.df['RecipeIngredientParts']

        # Kalori değişkeni nümeriğe çevriliyor ve nümerik olmayan değerler NaN ile dolduruluyor
        self.df['Calories'] = pd.to_numeric(self.df['Calories'], errors='coerce')
    
    def create_tfidf_matrix(self):
        """
        Text verisini nümerik veriye çevirmek için TF-IDF Vectorizer yöntemi kullanılmıştır
        Böylece her kelimenin önemi de yakalanmabilmiş oluyor.
        """
        self.tfidf = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.tfidf.fit_transform(self.df['combined_features'])
    
    def get_recommendations(self, query, n=5):
        """
        Verilen girdiye göre benzerlik hesaplayıp benzer tarifleri gösteren fonksiyon

        Params:
        ---------
        query (str): kullanıcı tarafından benzeri aranan yemek
        n (int): Döndürülecek tarif sayısı

        Return:
        ----------
        df (DataFrame): n kadar tarif detayları ile birlikte döndürülüyor
        """
        # Transform the query using the fitted TF-IDF vectorizer
        # Kullanıcı tarafından verilcek girdi oluşturulan tf-idf vectorizer ile nümeriğe çevrilir
        query_vec = self.tfidf.transform([query])
        
        # query ve bütün tarifler arasındaki benzerlik cosine similarity ile hesaplanır
        cosine_similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        
        # benzer bulunan indeksler alınır
        similar_indices = cosine_similarities.argsort()[:-n-1:-1]
        
        # Kullanıcıya isim, yapım aşamaları, malzemeler ve kaloriler gösterilir
        return self.df.iloc[similar_indices][['Name', 'RecipeInstructions', 'RecipeIngredientParts', 'Calories']]
    
    def get_recommendations_by_calories(self, min_cal, max_cal, query, n=5):
        """
        Kalori filtreleyerek verilen girdiye göre benzerlik hesaplayıp benzer tarifleri gösteren fonksiyon

        Params:
        ---------
        min_cal (int): istenilen minimum kalori
        max_cal (int): istenilen maksimum kalori
        query (str): kullanıcı tarafından benzeri aranan yemek
        n (int): Döndürülecek tarif sayısı

        Return:
        ----------
        df (DataFrame): n kadar tarif detayları ile birlikte döndürülüyor
        """
        # Kaloriye göre veride filtreleme yapılıyor
        calorie_filter = (self.df['Calories'] >= min_cal) & (self.df['Calories'] <= max_cal)
        filtered_df = self.df[calorie_filter]
        
        # Eğer filtrelenen veri boş ise uyarı dönüyor.
        if filtered_df.empty:
            return "No recipes found in the specified calorie range."
        
        else:
            # Eğer kullanıcı tarafından girdi verilmişse benzerlik hesaplanıyor
            query_vec = self.tfidf.transform([query])
            cosine_similarities = cosine_similarity(query_vec, self.tfidf.transform(filtered_df['combined_features'])).flatten()
            similar_indices = cosine_similarities.argsort()[:-n-1:-1]

        
        return filtered_df.iloc[similar_indices][['Name', 'RecipeInstructions', 'RecipeIngredientParts', 'Calories']]
    

# Usage example
recommender = RecipeRecommendationSystem('tarif.csv')

In [2]:
recommender.get_recommendations("Egg Sugar Lemon")

,Name,RecipeInstructions,RecipeIngredientParts,Calories
351298,Lemon Curd,"c(""Place butter into a metal bowl and set over...","c(""butter"", ""lemon"", ""fresh lemon juice"", ""sug...",391.1
349204,Lemon Loaf,"c(""Mix together the flour,sugar,baking powder,...","c(""flour"", ""sugar"", ""baking powder"", ""salt"", ""...",276.0
82513,Lemon Butter Sauce for Cake,"c(""Add all ingredients to a small sauce pan Co...","c(""butter"", ""sugar"", ""water"", ""egg"", ""lemon ju...",208.7
284893,Lemon Hard Sauce,"c(""Cream butter and salt."", ""Add egg sugar and...","c(""butter"", ""lemon rind"", ""lemon juice"", ""salt...",139.2
398056,Lemon Glazed Lemon Cookies,"c(""For the cookies: Preheat oven to 350 degree...","c(""all-purpose flour"", ""baking soda"", ""salt"", ...",127.9


In [3]:
recommender.get_recommendations_by_calories(150, 180, "Frozen blueberry Dessert")


,Name,RecipeInstructions,RecipeIngredientParts,Calories
494741,Blueberry Biscuits,"c(""Heat oven to 450°F Lightly grease cookie sh...","c(""self-rising flour"", ""butter"", ""sugar"", ""fro...",170.6
94631,Blueberry and Raspberry Freeze,"c(""Place all into a blender and blend till fro...","c(""frozen blueberries"", ""frozen raspberries"", ...",163.9
487639,The Blueberry and Straw Berry Muffins Ever,"c(""1• heat oven to 375'F."", ""2• grease 18 regu...","c(""butter"", ""margarine"", ""granulated sugar"", ""...",173.1
327788,Blueberry Frozen Yogurt Pie,"c(""To make crust:."", ""Preheat oven to 400*F."",...","c(""rolled oats"", ""dates"", ""honey"", ""fresh blue...",154.8
421277,Cold Blueberry Peach Soup,"""Measure out 10 ounces of each frozen fruit. ...","c(""frozen blueberries"", ""cinnamon"", ""sugar sub...",166.9
